Elasticsearch 

# Administracion y aspectos de Distribución

## Escalado horizontal
  - ES puede escalar horizontalmente para:
      - permitir cargas de consultas o de indexación más exigentes 
      - resiliencia

## *Cluster* 

 - Esta formado por uno o más *nodos*  
 - Un solo *nodo maestro* que se gestiona automáticamente 
 - Comparten el mismo nombre de cluster 
 - Distribuye la carga de consultas y los datos 
 - Hace transparente la búsqueda y combinacin de los resultados
 - Aún asi se puede consultar cada nodo por separado
 - No necesita Zookeper
 - Se pueden añadir nodos de manera dinámica

## *Master node*

- Gestiona el resto de nodos de un cluster -añadir nuevos o borrar
- Gestiona la creacion de índices - creacion y borrado

## Chequeando la salud del cluster

In [1]:
import requests

r = requests.get("http://localhost:9200/_cluster/health?pretty")
print r.text

{
  "cluster_name" : "ansible-es",
  "status" : "yellow",
  "timed_out" : false,
  "number_of_nodes" : 1,
  "number_of_data_nodes" : 1,
  "active_primary_shards" : 6,
  "active_shards" : 6,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 1,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 85.71428571428571
}



## Indices distribuidos

- Espacio donde se almacena los documentos de diferentes tipos
- ES puede tener multiples índices 
- Cada índice puede estar distribuido en varias máquinas  
  - ES hace 'transparente' la búsqueda e indexacion distribuida
- Cada índice se compone de varias particiones (*shards*)
- Cada *shard* puede tener varias réplicas

## *Shards*

 - Almacena los datos 
 - Cada shard es un único índice de Lucene
 - Proposito 
   - Añadir un mecanismo de redundancia logica 
   - Prevenir la perdida de datos en caso de fallo hardware
   

- Se puede definir el número de particiones por índice
 - Se puede definir el número de réplicas del índice - y cambiar de forma dinámica 

- *Primary shards* vs *replica shards* 

## *Scatter-Gather*
#### ¿Cómo se ejecuta una búsqueda distribuida? 

## Entendiendo los indicadores de salud del cluster

  - Verde
  - Amarillo -las replicas no están funcionando 
  - Rojo  

## Plugins de administración

  - [Marvel + Sense](https://www.elastic.co/products/marvel) - a partir de 2.0 es un plugin de Kibana
  - [Head](https://mobz.github.io/elasticsearch-head/)
  - [HQ](http://www.elastichq.org/index.html) 
  - [Kopf](https://github.com/lmenezes/elasticsearch-kopf)

## Cómo instalar plugins

- ES tiene una utilidad (*plugin*) que permite instalar plugins:
      - por nombre si se trata de un plugin oficial
      - desde un repo Github: username/repo/version
      - Maven central: groupId/artifactId/version
      - URL
- Se suele instalar en el directorio de ES bajo /bin
- ES ha crecido mucho en los 2 últimos años  - ¡mirar bien la matriz de versiones! 

<pre>
  cd /usr/share/elasticsearch
  sudo bin/plugin install <nombre>  # Para instalar
  sudo bin/plugin remove <nombre>   # Para quitar 
  sudo bin/plugin list              # Ver la lista de plugins
</pre>


## Plugins


[https://www.elastic.co/guide/en/elasticsearch/plugins/current/index.html](https://www.elastic.co/guide/en/elasticsearch/plugins/current/index.html)


## Configuración

 - Disponible en un fichero elasticsearch.yml (YAML) 
 - Situado en el directorio de ES o ruta típica del sistema: /etc/elasticsearch
 - Hay que configurar cada nodo de manera individual
 - Preconfigurado con parámetros típicos
 - **¡Cuidado!** En ES 1.x viene preconfigurado para unirse a un cluster por defecto
 

<pre>
##################### ElasticSearch Configuration Example #####################

# This file contains an overview of various configuration settings,
# targeted at operations staff. Application developers should
# consult the guide at <http://elasticsearch.org/guide>.
#
# The installation procedure is covered at
# <http://elasticsearch.org/guide/reference/setup/installation.html>.
#
# ElasticSearch comes with reasonable defaults for most settings,
# so you can try it out without bothering with configuration.
#
# Most of the time, these defaults are just fine for running a production
# cluster. If you're fine-tuning your cluster, or wondering about the
# effect of certain configuration option, please _do ask_ on the
# mailing list or IRC channel [http://elasticsearch.org/community].

# See <http://elasticsearch.org/guide/reference/setup/configuration.html>
# for information on supported formats and syntax for the configuration file.


################################### Cluster ###################################

# Cluster name identifies your cluster for auto-discovery. If you're running
# multiple clusters on the same network, make sure you're using unique names.
#
# cluster.name: elasticsearch
cluster.name: ansible-es

#################################### Node #####################################

# Node names are generated dynamically on startup, so you're relieved
# from configuring them manually. You can tie this node to a specific name:
#
# node.name: "Franz Kafka"
node.name: ansible-es-1

# Every node can be configured to allow or deny being eligible as the master,
# and to allow or deny to store the data.
#
# Allow this node to be eligible as a master node (enabled by default):
#
# node.master: true
#
# Allow this node to store data (enabled by default):
#
# node.data: true

# You can exploit these settings to design advanced cluster topologies.
#
# 1. You want this node to never become a master node, only to hold data.
#    This will be the "workhorse" of your cluster.
#
# node.master: false
# node.data: true
#
# 2. You want this node to only serve as a master: to not store any data and
#    to have free resources. This will be the "coordinator" of your cluster.
#
# node.master: true
# node.data: false
#
# 3. You want this node to be neither master nor data node, but
#    to act as a "search load balancer" (fetching data from nodes,
#    aggregating results, etc.)
#
# node.master: false
# node.data: false

# Use the Cluster Health API [http://localhost:9200/_cluster/health], the
# Node Info API [http://localhost:9200/_cluster/nodes] or GUI tools
# such as <http://github.com/lukas-vlcek/bigdesk> and
# <http://mobz.github.com/elasticsearch-head> to inspect the cluster state.

# A node can have generic attributes associated with it, which can later be used
# for customized shard allocation filtering, or allocation awareness. An attribute
# is a simple key value pair, similar to node.key: value, here is an example:
#
# node.rack: rack314

# By default, multiple nodes are allowed to start from the same installation location
# to disable it, set the following:
# node.max_local_storage_nodes: 1

# Disable network discovery by setting the following (useful for development):
# node.local: true

#################################### Index ####################################

# You can set a number of options (such as shard/replica options, mapping
# or analyzer definitions, translog settings, ...) for indices globally,
# in this file.
#
# Note, that it makes more sense to configure index settings specifically for
# a certain index, either when creating it or by using the index templates API.
#
# See <http://elasticsearch.org/guide/reference/index-modules/> and
# <http://elasticsearch.org/guide/reference/api/admin-indices-create-index.html>
# for more information.

# Set the number of shards (splits) of an index (5 by default):
#
# index.number_of_shards: 5
index.number_of_shards: 1

# Set the number of replicas (additional copies) of an index (1 by default):
#
# index.number_of_replicas: 1
index.number_of_replicas: 0

# Note, that for development on a local machine, with small indices, it usually
# makes sense to "disable" the distributed features:
#
# index.number_of_shards: 1
# index.number_of_replicas: 0

# These settings directly affect the performance of index and search operations
# in your cluster. Assuming you have enough machines to hold shards and
# replicas, the rule of thumb is:
#
# 1. Having more *shards* enhances the _indexing_ performance and allows to
#    _distribute_ a big index across machines.
# 2. Having more *replicas* enhances the _search_ performance and improves the
#    cluster _availability_.
#
# The "number_of_shards" is a one-time setting for an index.
#
# The "number_of_replicas" can be increased or decreased anytime,
# by using the Index Update Settings API.
#
# ElasticSearch takes care about load balancing, relocating, gathering the
# results from nodes, etc. Experiment with different settings to fine-tune
# your setup.

# Use the Index Status API (<http://localhost:9200/A/_status>) to inspect
# the index status.



#################################### Paths ####################################

# Path to directory containing configuration (this file and logging.yml):
#
# path.conf: /path/to/conf
path.conf: /etc/elasticsearch

# Path to directory where to store index data allocated for this node.
#
# path.data: /path/to/data
path.data: /var/lib/elasticsearch

# Can optionally include more than one location, causing data to be striped across
# the locations (a la RAID 0) on a file level, favouring locations with most free
# space on creation. For example:
#
# path.data: /path/to/data1,/path/to/data2

# Path to temporary files:
#
# path.work: /path/to/work
path.work: /tmp/elasticsearch

# Path to log files:
#
# path.logs: /path/to/logs
path.logs: /var/log/elasticsearch

# Path to where plugins are installed:
#
# path.plugins: /path/to/plugins
path.plugins: /usr/share/elasticsearch/plugins


# Path to snapshot/restore repost
#path.repo: //pppp


#################################### Plugin ###################################

# If a plugin listed here is not installed for current node, the node will not start.
#
# plugin.mandatory: mapper-attachments,lang-groovy


################################### Memory ####################################

# ElasticSearch performs poorly when JVM starts swapping: you should ensure that
# it _never_ swaps.
#
# Set this property to true to lock the memory:
#
#

# Make sure that the ES_MIN_MEM and ES_MAX_MEM environment variables are set
# to the same value, and that the machine has enough memory to allocate
# for ElasticSearch, leaving enough memory for the operating system itself.
#
# You should also make sure that the ElasticSearch process is allowed to lock
# the memory, eg. by using `ulimit -l unlimited`.





############################## Network And HTTP ###############################

# ElasticSearch, by default, binds itself to the 0.0.0.0 address, and listens
# on port [9200-9300] for HTTP traffic and on port [9300-9400] for node-to-node
# communication. (the range means that if the port is busy, it will automatically
# try the next port).

# Set the bind address specifically (IPv4 or IPv6):
#
# network.bind_host: 192.168.0.1

# Set the address other nodes will use to communicate with this node. If not
# set, it is automatically derived. It must point to an actual IP address.
#
# network.publish_host: 192.168.0.1

# Set both 'bind_host' and 'publish_host':
#
# network.host: 192.168.0.1
network.host: 0.0.0.0

# Set a custom port for the node to node communication (9300 by default):
#
# transport.tcp.port: 9300

# Enable compression for all communication between nodes (disabled by default):
#
# transport.tcp.compress: true

# Set a custom port to listen for HTTP traffic:
#
# http.port: 9200

# Set a custom allowed content length:
#
# http.max_content_length: 100mb

# Disable HTTP completely:
#
# http.enabled: false

# Basic HTTP


################################### Gateway ###################################

# The gateway allows for persisting the cluster state between full cluster
# restarts. Every change to the state (such as adding an index) will be stored
# in the gateway, and when the cluster starts up for the first time,
# it will read its state from the gateway.

# There are several types of gateway implementations. For more information,
# see <http://elasticsearch.org/guide/reference/modules/gateway>.

# The default gateway type is the "local" gateway (recommended):
#
# gateway.type: local

# Settings below control how and when to start the initial recovery process on
# a full cluster restart (to reuse as much local data as possible when using shared
# gateway).

# Allow recovery process after N nodes in a cluster are up:
#
# gateway.recover_after_nodes: 1

# Set the timeout to initiate the recovery process, once the N nodes
# from previous setting are up (accepts time value):
#
# gateway.recover_after_time: 5m

# Set how many nodes are expected in this cluster. Once these N nodes
# are up (and recover_after_nodes is met), begin recovery process immediately
# (without waiting for recover_after_time to expire):
#
# gateway.expected_nodes: 2

############################# Recovery Throttling #############################

# These settings allow to control the process of shards allocation between
# nodes during initial recovery, replica allocation, rebalancing,
# or when adding and removing nodes.

# Set the number of concurrent recoveries happening on a node:
#
# 1. During the initial recovery
#
# cluster.routing.allocation.node_initial_primaries_recoveries: 4

#
# 2. During adding/removing nodes, rebalancing, etc
#
# cluster.routing.allocation.node_concurrent_recoveries: 2

# Set to throttle throughput when recovering (eg. 100mb, by default unlimited):
#
# indices.recovery.max_size_per_sec: 0

# Set to limit the number of open concurrent streams when
# recovering a shard from a peer:
#
# indices.recovery.concurrent_streams: 5


################################## Discovery ##################################


# Discovery infrastructure ensures nodes can be found within a cluster
# and master node is elected. Multicast discovery is the default.

# Set to ensure a node sees N other master eligible nodes to be considered
# operational within the cluster. Set this option to a higher value (2-4)
# for large clusters (>3 nodes):
#
# discovery.zen.minimum_master_nodes: 1

# Set the time to wait for ping responses from other nodes when discovering.
# Set this option to a higher value on a slow or congested network
# to minimize discovery failures:
#
# discovery.zen.ping.timeout: 3s

# See <http://elasticsearch.org/guide/reference/modules/discovery/zen.html>
# for more information.

# Unicast discovery allows to explicitly control which nodes will be used
# to discover the cluster. It can be used when multicast is not present,
# or to restrict the cluster communication-wise.
#
# 1. Disable multicast discovery (enabled by default):
#
# discovery.zen.ping.multicast.enabled: false

#
# 2. Configure an initial list of master nodes in the cluster
#    to perform discovery when new nodes (master or data) are started:
#
# discovery.zen.ping.unicast.hosts: ["host1", "host2:port", "host3[portX-portY]"]

# EC2 discovery allows to use AWS EC2 API in order to perform discovery.
#
# You have to install the cloud-aws plugin for enabling the EC2 discovery.
#
# See <http://elasticsearch.org/guide/reference/modules/discovery/ec2.html>
# for more information.
#
# See <http://elasticsearch.org/tutorials/2011/08/22/elasticsearch-on-ec2.html>
# for a step-by-step tutorial.

# Fault Discovery
# See <http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/modules-discovery-zen.html#fault-detection>

# Configurations for Google Compute Engine plugin


################################## Slow Log ##################################

# Shard level query and fetch threshold logging.

#index.search.slowlog.threshold.query.warn: 10s
#index.search.slowlog.threshold.query.info: 5s
#index.search.slowlog.threshold.query.debug: 2s
#index.search.slowlog.threshold.query.trace: 500ms

#index.search.slowlog.threshold.fetch.warn: 1s
#index.search.slowlog.threshold.fetch.info: 800ms
#index.search.slowlog.threshold.fetch.debug: 500ms
#index.search.slowlog.threshold.fetch.trace: 200ms

#index.indexing.slowlog.threshold.index.warn: 10s
#index.indexing.slowlog.threshold.index.info: 5s
#index.indexing.slowlog.threshold.index.debug: 2s
#index.indexing.slowlog.threshold.index.trace: 500ms

################################## GC Logging ################################

#monitor.jvm.gc.ParNew.warn: 1000ms
#monitor.jvm.gc.ParNew.info: 700ms
#monitor.jvm.gc.ParNew.debug: 400ms

#monitor.jvm.gc.ConcurrentMarkSweep.warn: 10s
#monitor.jvm.gc.ConcurrentMarkSweep.info: 5s
#monitor.jvm.gc.ConcurrentMarkSweep.debug: 2s

################################### Varia #####################################


marvel.agent.enabled: false





################################### Dynamic Scripting #####################################

script.inline: true
script.indexed: true

################################### CORS Settings #####################################

http.cors.enabled: true

http.cors.allow-origin: http://localhost:5601




################################### Awareness Settings ##################################### 
</pre>

Paseo por [HQ](http://localhost:9200/_plugin/hq/) o Marvel

## Backup y restauración (Snapshot API)

 - Replicas permiten recuperarse de fallos en los nodos
 - No estan protegidos contra un fallo general => Backup!
 - Durante el desarrollo tambien es interesante realizar backups 
    - crear un índice puede ser costoso
    - reproducir condiciones específicas
    
##  Snapshot API 
 - disponible desde 
 - permite tomar instantaneas del índice y persistirlas a otro medio de almacenamiento (repositorio) 
 - permite recuperar las instantaneas y cargar un índice
 - Snapshots de uno o varios indices
 - Delta snapshopts
 
 ## Tipos de repositorios
  - Sistema de Ficheros (distribuido)  
  - Amazon S3
  - HDFS 
  - Azure Cloud 

### Definiendo un repositorio

### Haciendo un snapshot

### Recuperando un snapshot

## Alias de indices

  - En ES podemos definir alias de índices 
  - Permiten gestionar:
    - Cambiar un índice por otro de forma transparente
    - Agrupar multiples indices
    - Crear vistas en un subconjunto de documentos
     
   - Facilitan: 
      - cambios y pruebas en la indizacion - análisis, relevancia, etc
      - reindexar por un error de configiracion 
      - archivado de índices históricos - eficiencia
   

### Creando un alias

### Consultando los alias

In [ ]:
## *Warming up an index*